In [1]:
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Embedding
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.merge import Concatenate
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random

Using TensorFlow backend.


In [2]:
mode = '-nonstatic'
word_vectors = '-rand'

In [3]:
with open("mr.p", 'rb') as f:
    x = pickle.load(f, encoding='latin1')

# W - word2vec, W2 - rand
revs, W, W2, word_idx_map, vocab = x[0], x[1], x[2], x[3], x[4]
vocab_size = len(vocab)
print('Data successfully loaded')

if mode == "-nonstatic":
    print ("model architecture: CNN-non-static")
    non_static=True
elif mode=="-static":
    print ("model architecture: CNN-static")
    non_static=False
    
if word_vectors=="-rand":
    print ("using: random vectors")
    U = W2
elif word_vectors=="-word2vec":
    print ("using: word2vec vectors")
    U = W

Data successfully loaded
model architecture: CNN-non-static
using: random vectors


In [4]:
revs[0]['num_words']

34

In [5]:
max_sentence_len = -1

for i in range(len(revs)):
    num = revs[i]['num_words']
    max_sentence_len = max(num, max_sentence_len)
    
print('Max sentence length:', max_sentence_len)

Max sentence length: 56


In [6]:
word_idx_map[',']

9596

In [7]:
revs[100]

{'num_words': 13,
 'split': 4,
 'text': 'chicago is sophisticated , brash , sardonic , completely joyful in its execution',
 'y': 1}

In [8]:
N = len(revs)
N_train = int(np.round(N * 0.1))
N_test = int(N - N_train)

X = np.zeros((N, max_sentence_len), np.uint16)
Y = np.zeros((N,1), np.uint8)

random.shuffle(revs)
for i in range(len(revs)):
    list_wrds = revs[i]['text'].split()
    for j in range(len(list_wrds)):
        X[i][j] = word_idx_map[list_wrds[j]]
    Y[i] = revs[i]['y']

x_train = X[0:N_train]
y_train = Y[0:N_train]

x_test = X[N_train:N]
y_test = Y[N_train:N]

In [9]:
batch_size = 50
filter_sizes = [3,4,5]
num_filters = 100
dropout_prob = (0.5, 0.8)
hidden_dims = 50

l2_reg = 0.3
embedding_dim = 300

model_input = Input(shape= (max_sentence_len,))

z = Embedding(vocab_size+1, embedding_dim, input_length=max_sentence_len, name="embedding")(model_input)
z = Dropout(dropout_prob[0])(z)
    
z1 = Conv1D( filters=100, kernel_size=3, padding="valid", activation="relu", strides=1)(z)
z1 = MaxPooling1D(pool_size=2)(z1)
z1 = Flatten()(z1)

z2 = Conv1D( filters=100, kernel_size=4, padding="valid", activation="relu", strides=1)(z)
z2 = MaxPooling1D(pool_size=2)(z2)
z2 = Flatten()(z2)

z3 = Conv1D( filters=100, kernel_size=5, padding="valid", activation="relu", strides=1)(z)
z3 = MaxPooling1D(pool_size=2)(z3)
z3 = Flatten()(z3)

# Concatenate the output of all convolution layers
z = Concatenate()([z1, z2, z3])
z = Dropout(dropout_prob[1])(z)

z = Dense(hidden_dims, activation="relu")(z)
model_output = Dense(1, activation="sigmoid")(z)
    
model = Model(model_input, model_output)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 56)            0                                            
____________________________________________________________________________________________________
embedding (Embedding)            (None, 56, 300)       5629800                                      
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 56, 300)       0                                            
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, 54, 100)       90100                                        
___________________________________________________________________________________________

In [11]:
W.shape

(18766, 300)

In [12]:
embedding_layer = model.get_layer("embedding")
embedding_layer.set_weights([W])

model.fit(x_train, y_train, 
          batch_size = batch_size, 
          epochs=10,
          validation_data=(x_test, y_test), verbose=2)

Train on 1066 samples, validate on 9596 samples
Epoch 1/10
16s - loss: 0.6940 - acc: 0.5281 - val_loss: 0.6792 - val_acc: 0.5930
Epoch 2/10
5s - loss: 0.6416 - acc: 0.6435 - val_loss: 0.6293 - val_acc: 0.6700
Epoch 3/10
4s - loss: 0.5218 - acc: 0.7570 - val_loss: 0.6464 - val_acc: 0.6684
Epoch 4/10
4s - loss: 0.3614 - acc: 0.8311 - val_loss: 0.5444 - val_acc: 0.7332
Epoch 5/10
5s - loss: 0.2229 - acc: 0.9203 - val_loss: 0.5837 - val_acc: 0.7325
Epoch 6/10
5s - loss: 0.1487 - acc: 0.9456 - val_loss: 0.6824 - val_acc: 0.7281
Epoch 7/10
5s - loss: 0.1017 - acc: 0.9653 - val_loss: 0.7399 - val_acc: 0.7226
Epoch 8/10
5s - loss: 0.0733 - acc: 0.9756 - val_loss: 0.7583 - val_acc: 0.7354
Epoch 9/10
5s - loss: 0.0585 - acc: 0.9812 - val_loss: 0.7631 - val_acc: 0.7471
Epoch 10/10
5s - loss: 0.0506 - acc: 0.9822 - val_loss: 0.7981 - val_acc: 0.7442
